In [1]:
import numpy as np
import pandas as pd
import copy

In [2]:
def calculate_woe_iv(dataset):
    """
    对分箱后的特征计算WOE和IV
    :param dataset:DataFrame，计算数据,需要在特征分箱后的数据
    :return:
        iv: float，iv值
        df:DataFrame，woe和IV计算后结果

    Example
    -----------------------------------------------------------------
    >>> import random
    >>> data = pd.DataFrame([[random.random(),random.randint(0,1)] for _ in range(500)],columns=['feature','label'])
    >>> df = cut_width(dataset=data,inputcol='feature',labelcol='label',bins=10)
    >>> df.rename(columns={0:'neg',1:'pos'},inpalce=True)
    >>> iv, woe_iv_df = calculate_woe_iv(dataset=df)
    >>> iv
    0.037619588549634465
    >>> woe_iv_df
    label               neg  pos  pos_rate  neg_rate       woe        iv
    feature
    (-0.000313, 0.103]   23   27  0.104869  0.103004  0.017940  0.000033
    (0.103, 0.206]       23   27  0.104869  0.103004  0.017940  0.000033
    (0.206, 0.312]       29   21  0.082397  0.128755 -0.446365  0.020693
    (0.312, 0.418]       22   28  0.108614  0.098712  0.095591  0.000947
    (0.418, 0.535]       19   31  0.119850  0.085837  0.333793  0.011353
    (0.535, 0.614]       22   28  0.108614  0.098712  0.095591  0.000947
    (0.614, 0.705]       24   26  0.101124  0.107296 -0.059249  0.000366
    (0.705, 0.8]         24   26  0.101124  0.107296 -0.059249  0.000366
    (0.8, 0.891]         22   28  0.108614  0.098712  0.095591  0.000947
    (0.891, 0.991]       25   25  0.097378  0.111588 -0.136210  0.001936
    """
    df = copy.copy(dataset)
    df['pos_rate'] = (df['pos'] + 1) / df['pos'].sum()  # 计算每个分组内的响应（Y=1）占比，加1为了防止在计算woe时分子分母为0
    df['neg_rate'] = (df['neg'] + 1) / df['neg'].sum()  # 计算每个分组内的未响应（Y=0）占比
    df['woe'] = np.log(df['pos_rate'] / df['neg_rate'])  # 计算每个分组的WOE
    df['iv'] = (df['pos_rate'] - df['neg_rate']) * df['woe']  # 计算每个分组的IV
    iv = df['iv'].sum()
    return iv, df
    
def cut_width(dataset, inputcol, labelcol='label', bins=10):
    """
    等宽分箱
    :param dataset: DataFrame，计算数据
    :param inputcol: String,待分箱列列名
    :param labelcol: String,目标列列名
    :param bins: int,正整数，分箱数
    :return:
    :return:
        df: DataFrame，分箱后结果

    Example
    -----------------------------------------------------------------
    >>> import random
    >>> data = pd.DataFrame([[random.random(),random.randint(0,1)] for _ in range(500)],columns=['feature','label'])
    >>> df = cut_width(data,inputcol='feature',labelcol='label',bins=10)
    >>> df
        label                             good  bad
    feature
    (-0.0009308000000000001, 0.0968]    23   27
    (0.0968, 0.188]                     27   23
    (0.188, 0.29]                       25   25
    (0.29, 0.385]                       32   18
    (0.385, 0.472]                      31   19
    (0.472, 0.567]                      24   26
    (0.567, 0.686]                      24   26
    (0.686, 0.778]                      24   26
    (0.778, 0.912]                      26   24
    (0.912, 0.999]                      29   21
    """
    df = copy.copy(dataset)
    df[inputcol] = pd.qcut(x=df[inputcol], q=bins)
    df = pd.crosstab(index=df[inputcol], columns=df[labelcol], margins=False)
    return df




In [3]:
train_f = pd.read_csv("./data/train/feature.csv")
train_l = pd.read_csv("./data/train/label.csv")
df = pd.merge(train_f, train_l)

var_list = []
for i in range(10):
    for j in range(10):
        for k in range(10):
            var = 'v' + str(i) + str(j) + str(k)
            var_list.append(var)
df = cut_width(df,inputcol=var_list,labelcol='DEFAULT_LABEL',bins=10)
df

ValueError: Input array must be 1 dimensional

In [4]:
import pandas as pd, numpy as np, os, re, math, time
# to check monotonicity of a series 检验序列的单调性
def is_monotonic(temp_series):
    return all(temp_series[i] <= temp_series[i + 1] for i in range(len(temp_series) - 1)) or all(temp_series[i] >= temp_series[i + 1] for i in range(len(temp_series) - 1))

def prepare_bins(bin_data, c_i, target_col, max_bins):
    force_bin = True
    binned = False
    remarks = np.nan
    # ----------------- Monotonic binning -----------------
    for n_bins in range(max_bins, 2, -1):
        try:
            bin_data[c_i + "_bins"] = pd.qcut(bin_data[c_i], n_bins, duplicates="drop")
            monotonic_series = bin_data.groupby(c_i + "_bins")[target_col].mean().reset_index(drop=True)
            if is_monotonic(monotonic_series):
                force_bin = False
                binned = True
                remarks = "binned monotonically"
                break
        except:
            pass
    # ----------------- Force binning -----------------
    # creating 2 bins forcefully because 2 bins will always be monotonic
    if force_bin or (c_i + "_bins" in bin_data and bin_data[c_i + "_bins"].nunique() < 2):
        _min=bin_data[c_i].min()
        _mean=bin_data[c_i].mean()
        _max=bin_data[c_i].max()
        bin_data[c_i + "_bins"] = pd.cut(bin_data[c_i], [_min, _mean, _max], include_lowest=True)
        if bin_data[c_i + "_bins"].nunique() == 2:
            binned = True
            remarks = "binned forcefully"
    
    if binned:
        return c_i + "_bins", remarks, bin_data[[c_i, c_i+"_bins", target_col]].copy()
    else:
        remarks = "couldn't bin"
        return c_i, remarks, bin_data[[c_i, target_col]].copy()

# calculate WOE and IV for every group/bin/class for a provided feature 计算所提供功能的每个组/箱/类的WOE和IV
def iv_woe_4iter(binned_data, target_col, class_col):
    if "_bins" in class_col:
        binned_data[class_col] = binned_data[class_col].cat.add_categories(['Missing'])
        binned_data[class_col] = binned_data[class_col].fillna("Missing")
        temp_groupby = binned_data.groupby(class_col).agg({class_col.replace("_bins", ""):["min", "max"],
                                                           target_col: ["count", "sum", "mean"]}).reset_index()
    else:
        binned_data[class_col] = binned_data[class_col].fillna("Missing")
        temp_groupby = binned_data.groupby(class_col).agg({class_col:["first", "first"],
                                                           target_col: ["count", "sum", "mean"]}).reset_index()
    
    temp_groupby.columns = ["sample_class", "min_value", "max_value", "sample_count", "event_count", "event_rate"]
    temp_groupby["non_event_count"] = temp_groupby["sample_count"] - temp_groupby["event_count"]
    temp_groupby["non_event_rate"] = 1 - temp_groupby["event_rate"]
    temp_groupby = temp_groupby[["sample_class", "min_value", "max_value", "sample_count",
                                 "non_event_count", "non_event_rate", "event_count", "event_rate"]]
    
    if "_bins" not in class_col and "Missing" in temp_groupby["min_value"]:
        temp_groupby["min_value"] = temp_groupby["min_value"].replace({"Missing": np.nan})
        temp_groupby["max_value"] = temp_groupby["max_value"].replace({"Missing": np.nan})
    temp_groupby["feature"] = class_col
    if "_bins" in class_col:
        temp_groupby["sample_class_label"]=temp_groupby["sample_class"].replace({"Missing": np.nan}).astype('category').cat.codes.replace({-1: np.nan})
    else:
        temp_groupby["sample_class_label"]=np.nan
    temp_groupby = temp_groupby[["feature", "sample_class", "sample_class_label", "sample_count", "min_value", "max_value",
                                 "non_event_count", "non_event_rate", "event_count", "event_rate"]]
    
    """
    **********get distribution of good and bad 得到好的和坏的分布
    """
    temp_groupby['distbn_non_event'] = temp_groupby["non_event_count"]/temp_groupby["non_event_count"].sum()
    temp_groupby['distbn_event'] = temp_groupby["event_count"]/temp_groupby["event_count"].sum()

    temp_groupby['woe'] = np.log(temp_groupby['distbn_non_event'] / temp_groupby['distbn_event'])
    temp_groupby['iv'] = (temp_groupby['distbn_non_event'] - temp_groupby['distbn_event']) * temp_groupby['woe']
    
    temp_groupby["woe"] = temp_groupby["woe"].replace([np.inf,-np.inf],0)
    temp_groupby["iv"] = temp_groupby["iv"].replace([np.inf,-np.inf],0)
    
    return temp_groupby

"""
- iterate over all features. 迭代所有功能。
- calculate WOE & IV for there classes.计算这些类别的woe与iv值
- append to one DataFrame woe_iv.追加到数据帧woe_iv中。
"""
def var_iter(data, target_col, max_bins):
    woe_iv = pd.DataFrame()
    remarks_list = []
    for c_i in data.columns:
        if c_i not in [target_col]:
            # check if binning is required. if yes, then prepare bins and calculate woe and iv.
            """
            ----logic---
            binning is done only when feature is continuous and non-binary. 仅当数据是连续型，且不是二进制时才会处理
            Note: Make sure dtype of continuous columns in dataframe is not object. 确保dataframe中连续列的数据类型不是object。
            """
            c_i_start_time=time.time()
            if np.issubdtype(data[c_i], np.number) and data[c_i].nunique() > 2:
                class_col, remarks, binned_data = prepare_bins(data[[c_i, target_col]].copy(), c_i, target_col, max_bins)
                agg_data = iv_woe_4iter(binned_data.copy(), target_col, class_col)
                remarks_list.append({"feature": c_i, "remarks": remarks})
            else:
                agg_data = iv_woe_4iter(data[[c_i, target_col]].copy(), target_col, c_i)
                remarks_list.append({"feature": c_i, "remarks": "categorical"})
            # print("---{} seconds. c_i: {}----".format(round(time.time() - c_i_start_time, 2), c_i))
            woe_iv = woe_iv.append(agg_data)
    return woe_iv, pd.DataFrame(remarks_list)

# after getting woe and iv for all classes of features calculate aggregated IV values for features.
def get_iv_woe(data, target_col, max_bins):
    func_start_time = time.time()
    woe_iv, binning_remarks = var_iter(data, target_col, max_bins)
    print("------------------IV and WOE calculated for individual groups.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    
    woe_iv["feature"] = woe_iv["feature"].replace("_bins", "", regex=True)    
    woe_iv = woe_iv[["feature", "sample_class", "sample_class_label", "sample_count", "min_value", "max_value",
                     "non_event_count", "non_event_rate", "event_count", "event_rate", 'distbn_non_event',
                     'distbn_event', 'woe', 'iv']]
    
    iv = woe_iv.groupby("feature")[["iv"]].agg(["sum", "count"]).reset_index()
    print("------------------Aggregated IV values for features calculated.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    
    iv.columns = ["feature", "iv", "number_of_classes"]
    null_percent_data=pd.DataFrame(data.isnull().mean()).reset_index()
    null_percent_data.columns=["feature", "feature_null_percent"]
    iv=iv.merge(null_percent_data, on="feature", how="left")
    print("------------------Null percent calculated in features.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    iv = iv.merge(binning_remarks, on="feature", how="left")
    woe_iv = woe_iv.merge(iv[["feature", "iv", "remarks"]].rename(columns={"iv": "iv_sum"}), on="feature", how="left")
    print("------------------Binning remarks added and process is complete.------------------")
    print("Total time elapsed: {} minutes".format(round((time.time() - func_start_time) / 60, 3)))
    return iv, woe_iv.replace({"Missing": np.nan})

In [14]:
train_f = pd.read_csv("./data/train/feature.csv")
train_l = pd.read_csv("./data/train/label.csv")
data = pd.merge(train_f, train_l)
data.rename(columns={'APPLICATION_DATE':'APPLICATION_DATE_bins'}, inplace = True)

iv, woe_iv = get_iv_woe(data.copy(), target_col="DEFAULT_LABEL", max_bins=20)
print(iv.shape, woe_iv.shape)

/home/ubuntu/anaconda3/envs/cd80test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ubuntu/anaconda3/envs/cd80test/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/ubuntu/anaconda3/envs/cd80test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ubuntu/anaconda3/envs/cd80test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/ubuntu/anaconda3/envs/cd80test/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: PerformanceWar

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
woe_iv[woe_iv['feature'] == 'v010']

In [9]:
woe_iv.to_csv("woe_result.csv")